In [20]:
#! pip install ipywidgets
#! jupyter nbextension enable --py widgetsnbextension --sys-prefix
from ipywidgets import interact
from ipywidgets.widgets import FloatText, BoundedIntText, FloatSlider, IntSlider, IntRangeSlider, fixed, Text

In [15]:
# Widgets for interactive functions / plots
style = {'description_width': 'initial'}

debt_text = FloatText(value=100000, step=100, description='Debt:')

payoff_slider = FloatSlider(value=1000, step=10, min=500, max=10_000, description='Monthly pay off:', style=style)

interest_slider = FloatSlider(value=0.15, step=0.01, min=0, max=0.25, description='Interest:')
interest_text = Text(value='0.05, 0.05, 0.05, 0.05, 0.05, 0.05', description="Interests")

tax_slider = FloatSlider(value=0.19, step=0.01, min=0, max=0.30, description='Tax:')

months_slider = IntSlider(value=1, step=1, min=1, max=360, description='Months:')
years_range_slider = IntRangeSlider(value=(1, 40), step=1, min=1, max=100, description='Years:')

# Loans

### Mortgage

We know already that if we have initial deposit $D_0$ and different contribution per compound $c$ then the general formula for $D_n$:
\begin{equation}
$D_n = D_0 q^{nm} + c \sum_{i=0}^{nm-1} q^i = D_0 q^{nm} + c \cdot \frac{1 - q^{nm}}{1-q}$.
\end{equation}

Deposits and debts are actually the same thing -- there are just different sides of contract. In deposit you loan money to the bank and in return get some interests, just like with debt. This is why we can use formulas from deposits, but we need to change initial deposit to be negative, as to represent our debt and contributions will be added to negative amount so that we are getting closer to paying it of.

In [11]:
def debt(debt: float, n: int, payoff: float, interest: float, tax: float = 0.19):
    """Calculate debt amount after n compound periods."""
    r = interest / 12 * (1-tax)
    d_n = (debt * (1 + r) ** n + payoff * (1 - (1+r)**n) / (-r))
    return d_n

In [15]:
def debt_test(debt: float, n: int, payoff: float, interest: float, tax: float = 0.19):
    """Compute debt amount after n compound periods."""
    d_n = debt
    r = interest / 12 * (1-tax)
    for i in range(n):
        d_n = d_n * (1+r) + payoff
        print(f'period={i+1} with payoff, debt={d_n:.2f}')
    return d_n

d1 = debt(debt=-100_000, n=120, payoff=1443.39, interest=0.15)
d2 = debt_test(debt=-100_000, n=120, payoff=1443.39, interest=0.15)

period=1 with payoff, debt=-99569.11
period=2 with payoff, debt=-99133.86
period=3 with payoff, debt=-98694.20
period=4 with payoff, debt=-98250.09
period=5 with payoff, debt=-97801.48
period=6 with payoff, debt=-97348.33
period=7 with payoff, debt=-96890.59
period=8 with payoff, debt=-96428.22
period=9 with payoff, debt=-95961.16
period=10 with payoff, debt=-95489.38
period=11 with payoff, debt=-95012.82
period=12 with payoff, debt=-94531.43
period=13 with payoff, debt=-94045.18
period=14 with payoff, debt=-93553.99
period=15 with payoff, debt=-93057.84
period=16 with payoff, debt=-92556.66
period=17 with payoff, debt=-92050.40
period=18 with payoff, debt=-91539.02
period=19 with payoff, debt=-91022.47
period=20 with payoff, debt=-90500.68
period=21 with payoff, debt=-89973.61
period=22 with payoff, debt=-89441.20
period=23 with payoff, debt=-88903.40
period=24 with payoff, debt=-88360.16
period=25 with payoff, debt=-87811.42
period=26 with payoff, debt=-87257.12
period=27 with payoff

## Amount of payoff
We are interested in the amount of payoff per period $c$ given initial debt $D_0$, $p\%$ interest rate, $s\%$ tax fee and $m=12$ frequency of compounding. Let's denote $q = 1 +r, \; r = \frac{p}{m} \cdot (1-s)$ To get formula for it, we need to know the number of periods $N=nm$ we want to pay off debt. Paying off the debt in $N$ periods mean we have equation $D_n = 0$:

\begin{align}
    D_0 q^{nm} + c \cdot \frac{1 - q^N}{1-q} = 0 \\
    c = \frac{-D_0 q^N (1-q)}{1-q^N}
\end{align}




In [18]:
def payoff(debt: float, N: int, interest: float, tax: float = 0.19):
    q = 1 + interest / 12 * (1-tax)
    return debt * q**N * (1 - q) / (1 - q**N)

payoff(debt=100_000, N=120, interest=0.15)

1890.565790821865

In [17]:
def payoff_print(debt: float, N: int, interest: float, tax: float = 0.19):
    return f'Pay off monthly: {payoff(debt, N, interest, tax):.2f}'
interact(payoff_print, debt=debt_text, N=months_slider, interest=interest_slider, tax=tax_slider);

interactive(children=(FloatText(value=100000.0, description='Debt:', step=100.0), IntSlider(value=87, descript…

## Number of periods
We are interested in the number of periods $N$ we need to pay off the debt $D_0$, with $p\%$ interest rate, $s\%$ tax fee and $m=12$ frequency of compounding. Let's denote $q = 1 +r, \; r = \frac{p}{m} \cdot (1-s)$ To get formula for it, we need to know the amount of money we are willing to pay monthly. Using the same equation as before:

\begin{align}
    D_0 q^{nm} + c \cdot \frac{1 - q^N}{1-q} = 0 \\
    N = \log_q (c) - \log_q \left( -D_0(1-q) + c \right)
\end{align}

In [13]:
from math import log
def num_months(debt: float, payoff: float, interest: float, tax: float = 0.19):
    q = 1 + interest / 12 * (1-tax)
    return log(payoff, q) - log(debt * (1-q) + payoff, q)

num_months(debt=100_000, payoff=1443.394, interest=0.15)

120.00001737331263

In [14]:
def num_months_print(debt: float, payoff: float, interest: float, tax: float = 0.19):
    return f'Months : {num_months(debt, payoff, interest, tax):.2f}'
interact(num_months_print, debt=debt_text, payoff=payoff_slider, interest=interest_slider, tax=tax_slider);

interactive(children=(FloatText(value=100000.0, description='Debt:', step=100.0), FloatSlider(value=1000.0, de…